In [ ]:

# Install and configure the Kaggle API
!pip install kaggle

# Ensure you have the Kaggle API key file (kaggle.json) in your ~/.kaggle directory or set it up here
import os
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Specify the dataset to download
dataset = "msambare/fer2013"  # FER2013 dataset
download_path = "./datasets"  # Directory to save the dataset

# Download and unzip the dataset
api.dataset_download_files(dataset, path=download_path, unzip=True)
print(f"Dataset downloaded and extracted to: {os.path.abspath(download_path)}")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import os
import glob
import pickle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, SpatialDropout2D, BatchNormalization, Input,Activation, Dense, Flatten ,MaxPooling2D
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping,ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.models import Model,save_model
from sklearn.metrics import accuracy_score, confusion_matrix,ConfusionMatrixDisplay

In [ ]:
path  = "/kaggle/input/fer2013/"

In [ ]:
image_list = os.listdir(path)
image_list

In [ ]:
image_filenames = glob.glob(os.path.join(path,'**/*.jpg'), recursive=True)

# create an info dataframe
df = pd.DataFrame({'filename': image_filenames})
df['partition'] = df['filename'].apply(lambda x: x.split(os.sep)[-3])
df['label'] = df['filename'].apply(lambda x: x.split(os.sep)[-2])

df

In [ ]:
fig, axs = plt.subplots(7, 5, figsize=(20,20))
for i in range(5):
    axs.flat[i].imshow(plt.imread( df.query('label == "angry"')['filename'].iloc[i]))
    axs.flat[i].set_title(str("angry"))
    axs.flat[i].axis('off')
    
for i in range(5):
    axs.flat[i+5].imshow(plt.imread( df.query('label == "disgust"')['filename'].iloc[i]))
    axs.flat[i+5].set_title(str("disgust"))
    axs.flat[i+5].axis('off')

for i in range(5):
    axs.flat[i+10].imshow(plt.imread( df.query('label == "fear"')['filename'].iloc[i]))
    axs.flat[i+10].set_title(str("fear"))
    axs.flat[i+10].axis('off')

for i in range(5):
    axs.flat[i+15].imshow(plt.imread( df.query('label == "happy"')['filename'].iloc[i]))
    axs.flat[i+15].set_title(str("happy"))
    axs.flat[i+15].axis('off')
    
for i in range(5):
    axs.flat[i+20].imshow(plt.imread( df.query('label == "neutral"')['filename'].iloc[i]))
    axs.flat[i+20].set_title(str("neutral"))
    axs.flat[i+20].axis('off')
    
for i in range(5):
    axs.flat[i+25].imshow(plt.imread( df.query('label == "sad"')['filename'].iloc[i]))
    axs.flat[i+25].set_title(str("sad"))
    axs.flat[i+25].axis('off')
    
for i in range(5):
    axs.flat[i+30].imshow(plt.imread( df.query('label == "surprise"')['filename'].iloc[i]))
    axs.flat[i+30].set_title(str("surprise"))
    axs.flat[i+30].axis('off')
    
plt.tight_layout()
plt.show()


In [ ]:
image_generator=ImageDataGenerator(rescale=1/255)


Training = image_generator.flow_from_directory('/kaggle/input/fer2013/train',target_size=(48,48),
                                            batch_size=64,class_mode='categorical',shuffle=True)


Test = image_generator.flow_from_directory('/kaggle/input/fer2013/test' ,target_size=(48,48),
                                            batch_size=64,class_mode='categorical',shuffle=True)

In [ ]:
df['label'].value_counts(ascending=True)

In [ ]:
print(Training.class_indices)
print(Test.class_indices)

In [ ]:
classes = 7

model = Sequential()

#1st CNN layer
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.3))

#2nd CNN layer
model.add(Conv2D(128,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.3))

#3rd CNN layer
model.add(Conv2D(512,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout (0.3))

#4th CNN layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

model.add(Flatten())

#Fully connected 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))


# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(classes, activation='softmax'))



opt = Adam(lr = 0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
train_step=Training.n//Training.batch_size
test_step=Test.n//Test.batch_size


num_epochs = 20

#checkpoint = ModelCheckpoint("./model.h5", monitor='val_acc', verbose=1, save_best_only=True, mode='max')

reduce_learningrate = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

early_stopping = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience= 4,
                          verbose=1,
                          restore_best_weights=True
                          )

model_checkpoint = ModelCheckpoint('best_model_1.h5',monitor="val_accuracy",verbose=1,
                                save_best_only=True,mode="max")

callbacks_list = [early_stopping,reduce_learningrate,model_checkpoint]

history = model.fit_generator(Training,
                                      steps_per_epoch=train_step,
                                      epochs=num_epochs, 
                                      validation_data=Test,
                                      validation_steps=test_step,
                                      callbacks=callbacks_list) 

In [ ]:
score = model.evaluate(Test)
print(score)

In [ ]:
print(Test.filepaths[1000])
print('--------------------------------------')
print(Test.classes[1000])

In [ ]:
images,labels=Test.next()
pred=np.argmax(model.predict(tf.expand_dims(images[1],axis=0)))

ls = list(Test.class_indices.keys())

plt.imshow(images[1])
plt.title(ls[pred])

In [ ]:
plt.plot(history.history['accuracy'],label='training')
plt.plot(history.history['val_accuracy'],label='validation')
plt.plot(history.history["loss"],label='loss')
plt.plot(history.history["val_loss"],label='val_loss')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
preds = model.predict(Test)
y_pred = np.argmax(preds , axis = 1 )


In [ ]:
y_pred

In [ ]:
y_pred = model.predict_generator(Test)
y_pred_classes = np.argmax(y_pred, axis = 1)

ConfusionMatrixDisplay(confusion_matrix(Test.classes,y_pred_classes),
                       display_labels=list(Training.class_indices.keys())).plot()

In [ ]:
model.save('my_model.h5')

In [ ]:
# Loading the best fit model 
from keras.models import load_model
model = load_model("my_model.h5")
model.summary()